<a href="https://colab.research.google.com/github/SATHYARANGARAJ53/medicine_inventory/blob/main/projectmodel_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import datetime

# Load CSV file
def load_data(file_path):
    df = pd.read_csv("Medicine_Details.csv")
    return df

# Extract data for the current month from past six years
def extract_relevant_data(df):
    current_month = datetime.datetime.now().strftime('%B_%Y')  # Example: 'February_2025'
    years = [(datetime.datetime.now().year - i) for i in range(1, 7)]

    month_years = [current_month.replace(str(datetime.datetime.now().year), str(year)) for year in years]

    # Filter relevant columns
    relevant_cols = ['Medicine Name'] + month_years + ['Stock Level']
    df_filtered = df[relevant_cols].dropna()
    return df_filtered, month_years

# Prepare data for LSTM
def prepare_data(df, month_years):
    scaler = MinMaxScaler()
    X = df[month_years].values  # Past six years' data
    y = df[month_years[-1]].values  # Predicting this year’s consumption
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y.reshape(-1, 1))

    # Reshape for LSTM (samples, time steps, features)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    return X, y, scaler

# Define LSTM model
def build_lstm_model():
    model = Sequential([
        LSTM(50, activation='relu', input_shape=(6, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Train and evaluate model
def train_and_predict(file_path):
    df = load_data(file_path)
    df_filtered, month_years = extract_relevant_data(df)
    X, y, scaler = prepare_data(df_filtered, month_years)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_lstm_model()

    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=1, validation_data=(X_test, y_test))

    # Predict current month's consumption
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    # Create a DataFrame with predictions and corresponding medicines
    predictions_df = pd.DataFrame(predictions, columns=['Predicted_Consumption'])
    predictions_df['Medicine Name'] = df_filtered.iloc[X_test.shape[0]:]['Medicine Name'].reset_index(drop=True)

    # Return predictions along with medicine names
    return predictions_df[['Medicine Name', 'Predicted_Consumption']]

    # Predict current month's consumption
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    # Return predictions along with medicine names
    df_filtered['Predicted_Consumption'] = predictions
    return df_filtered[['Medicine Name', 'Predicted_Consumption']]

# Example usage
file_path = "Medicine_Details.csv"  # Replace with your actual file
predicted_df = train_and_predict(file_path)
print(predicted_df)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 200ms/step - loss: 0.3258 - val_loss: 0.2129
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2717 - val_loss: 0.1883
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.2379 - val_loss: 0.1662
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.2403 - val_loss: 0.1450
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - loss: 0.2230 - val_loss: 0.1239
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.1659 - val_loss: 0.1026
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.1450 - val_loss: 0.0808
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.1350 - val_loss: 0.0628
Epoch 9/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0949 - val_loss: 0.0578
Epoch 10/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0837 - val_loss: 0.0632
Epoch 11/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0854 - val_loss: 0.0602
Epoch 12/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0833 - val_loss: 0.0534
Epoch 13/5